# PMR3508 - Aprendizado de Máquina e Reconhecimento de Padrões

Testing kNN with adult database from UCI repository.

**Author:** Lucas Hideki Takeuchi Okamura

**Hash:** 50

## 1. Imports

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from scikitplot import metrics as mt

from sklearn.preprocessing import RobustScaler, StandardScaler, OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, classification_report,recall_score,f1_score,roc_auc_score, plot_precision_recall_curve, precision_score,roc_curve
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

from scipy.stats import loguniform as sp_loguniform

import warnings
warnings.filterwarnings('ignore')

## 2. Loading Dataset

In [ ]:
train = pd.read_csv('../input/adult-pmr3508/train_data.csv', 
                    sep=r'\s*,\s*',
                    engine='python',
                    na_values="?")
test = pd.read_csv('../input/adult-pmr3508/test_data.csv', 
                   sep=r'\s*,\s*',
                    engine='python',
                    na_values="?")

In [ ]:
train.head()

In [ ]:
test.head()

We see from the datasets that the income column is the target of the problem and that's what we want to predict.

## 3. Data Description

In this section we will see a overview of the datasets, considering these subsections:

* Data Dimensions
* Data Types
* Check missing values
* Fillout missing values
* Descriptive Statistical

### 3.1. Data Dimensions

In [ ]:
print(f"Training data shape: {train.shape}")
print(f"Testing data shape: {test.shape}")

### 3.2. Data Types

We can see that the income label is categorical, thus we have a classification problem.

In [ ]:
train.dtypes

### 3.3. Check and fill missing values

From the training data, there are missing data in occupation, workclass and native.country. We will fill them out with their most common values.

In [ ]:
train.isna().sum().sort_values(ascending = False)

In [ ]:
test.isna().sum().sort_values(ascending = False)

In [ ]:
# fill nans
occupation_mc = train.loc[:, 'occupation'].mode()[0]
workclass_mc = train.loc[:, 'workclass'].mode()[0]
native_country_mc = train.loc[:, 'native.country'].mode()[0]

print(f'occupation most common attribute: {occupation_mc}')
print(f'workclass most common attribute: {workclass_mc}')
print(f'native.country most common attribute: {native_country_mc}')

train.loc[:, 'occupation'].fillna(occupation_mc, inplace = True)
train.loc[:, 'workclass'].fillna(workclass_mc, inplace = True)
train.loc[:, 'native.country'].fillna(native_country_mc, inplace = True)

test.loc[:, 'occupation'].fillna(occupation_mc, inplace = True)
test.loc[:, 'workclass'].fillna(workclass_mc, inplace = True)
test.loc[:, 'native.country'].fillna(native_country_mc, inplace = True)

In [ ]:
# checking missing values again - train
train.isna().sum().sort_values(ascending = False)

In [ ]:
# checking missing values again - test
test.isna().sum().sort_values(ascending = False)

### 3.4. Descriptive statistical

In this section we will have a statistical overview in the training dataset, considering numerical and categorical variables.

In [ ]:
num_attributes = train.select_dtypes(include = ['int64','float64']).drop(columns = 'Id')
cat_attributes = train.select_dtypes(exclude = ['int64','float64'])

#### 3.4.1. Numerical Attributes

In [ ]:
num_attributes.describe()

Although education.num is a numeric variable, it represents the category relative to the person education level, so it will be treated as a categorical variable.

#### 3.4.2. Categorical Attributes

Here we can see how many categories each variable has, showing that native.country is the feature with most categories.

In [ ]:
# inserting education.num to categorical variables
new_cat_variables = list(cat_attributes.columns) + ['education.num']
cat_attributes = train[new_cat_variables].copy()

# removing education.num from numerical variables
num_attributes = num_attributes.drop(columns = 'education.num')

In [ ]:
cat_attributes.apply(lambda x: x.unique().shape[0])

## 4. Exploratory Data Analysis

Here the data characteristics will be summarized and understood from plots and statistical inferences.

### 4.1. Univariate Analysis

#### 4.1.1. Response Variable (income)

Plotting a barplot to understand how the target variable is distributed. It is possible to see that the training dataset is unbalanced relative to the target labels, as the people who have a income <=50K is the majority when compared to those who have income >50K.

In [ ]:
_ = sns.catplot(x = 'income', kind = 'count', data = train)

#### 4.1.2. Numerical Variable

We can infere some informations from the graphs:

* **age**: most part of the people have less than 60 years
* **fnlwgt**: almost all the data concentrated in values less than $0.75 * 10^{6}$, despite some outliers
* **capital.gain** and **capital.loss**: values are concentrated near 0, despite some outliers
* **hours.per.week**: values are concentrated near 40, an it has a lot of outliers

In [ ]:
fig, ax = plt.subplots(1, 5, figsize = (25,7))
for index, column in enumerate(num_attributes.columns):
    _ = sns.boxplot(y = num_attributes[column], ax = ax[index], orient = 'v')
    
_ = num_attributes.hist(bins = 25, figsize = (20, 7))
_ = plt.tight_layout()

#### 4.1.3. Categorical Variable

From the data, we can infere:

* **workclass**: the majority of the public studied in private schools
* **education**: the most significant categories are HS-grad, Bachelors and Some College
* **marital.status**: Married-civ-spouse and Never-married are predominant
* **occupation**: this features has a well distributed data
* **relationship**: Husband predominates
* **race**: most part of the public is white
* **sex**: more men were asked than women
* **native.country**: clear predominance of american people
* **education.num**: class 9 predominates

In [ ]:
fig, axes = plt.subplots(5, 2, figsize = (30,15), constrained_layout=True)
axes = axes.flatten()

for index, column in enumerate(cat_attributes.columns):
    _ = sns.countplot(cat_attributes[column], ax = axes[index], order = cat_attributes[column].value_counts().index)
    axes[index].tick_params(axis='x', rotation=90)

### 4.2. Bivariate Analysis

Now, some features will be evaluated relative to their relation to the target variable income.

#### 4.2.1. sex - income comparison

From the graphs we see that the proportion between men and woman is different when the income is considered. While there are, proportionaly more men than women in the >50K class, this proportion reduces when the class <=50K is considered, evidencing that men are more prone to have a income bigger than 50K.

In [ ]:
# separating publics by income
less_income = train[train['income'] == '<=50K']
less_income = less_income[['sex','income']].groupby("sex").count().reset_index()

more_income = train[train['income'] == '>50K']
more_income = more_income[['sex','income']].groupby("sex").count().reset_index()

# plot 1 - income <=50K
plt.subplot(1,2,1)
_ = sns.barplot(x = 'sex', y = 'income', data = less_income)
_ = plt.title("Income <=50K")

#plot2 - gender x cardio (bar plot)
plt.subplot(1,2,2)
_ = sns.barplot(x = 'sex', y = 'income', data = more_income)
_ = plt.title("Income >50K")

_ = plt.tight_layout()

#### 4.2.2. age - income comparison

Although the correlation between age and income is quite low, it is possible to see a slight trend that older people have higher incomes.

In [ ]:
# plot 1 - boxplot
plt.subplot(1,2,1)
_ = sns.boxplot(x = 'income', y = 'age', data = train)
_ = plt.title("Age x Income")

# plot 2 - correlation
plt.subplot(1,2,2)
df_corr = train[['age', 'income']]
df_corr['income'] = df_corr['income'].map({'<=50K' : 0, '>50K' : 1})
_ = sns.heatmap(df_corr[['age', 'income']].corr(method = 'pearson'), annot = True)

#### 4.2.3. education - income comparison

From the data, we see that the majority of people with income less than 50K has only completed High School or some college, while people with bachelor dominates in the higher income public.

In [ ]:
# separating publics by income
less_income = train[train['income'] == '<=50K']
less_income = less_income[['education','income']].groupby("education").count().sort_values('income').reset_index()

more_income = train[train['income'] == '>50K']
more_income = more_income[['education','income']].groupby("education").count().sort_values('income').reset_index()

fig, axes = plt.subplots(1,2,figsize = (25, 5))

# plot 1 - income <=50K
_ = sns.barplot(x = 'education', y = 'income', data = less_income, ax = axes[0])
_ = axes[0].set_title("Income <=50K")
_ = axes[0].tick_params(axis='x', rotation=90)

#plot2 - gender x cardio (bar plot)
_ = sns.barplot(x = 'education', y = 'income', data = more_income, ax = axes[1])
_ = axes[1].set_title("Income >50K")
_ = axes[1].tick_params(axis='x', rotation=90)

_ = plt.tight_layout()

### 4.3. Multivariate Analysis

From the multivariate analysis, we look for features that are not correlated to others features and features that are somehow correlated to the target, in order to predict the result more accurately basing on those correlations. From the heatmap, we see that none of the features have a high correlation to each other and age, education.num, capital.gain and hours.per.week have decent correlation to the target variable income, indicating that they are good variables to the model.

In [ ]:
#exclude non-numerical variables
df_corr = train.copy()
df_corr['income'] = df_corr['income'].map({'<=50K' : 0, '>50K' : 1})
aux1 = df_corr.select_dtypes(exclude = ['object'])

#plot correlation plot to our numerical variables
fig, ax = plt.subplots(figsize = (20, 8))
_ = sns.heatmap(aux1.corr(method = 'pearson'), annot = True, ax = ax)

## 5. Feature Engineering

We will create a variable called "capital", which is calculated by capital.gain - capital.loss:

In [ ]:
train['capital'] = train['capital.gain'] - train['capital.loss']
test['capital'] = test['capital.gain'] - test['capital.loss']

In [ ]:
train[['capital']].describe()

## 6. Feature Selection

Since the majority of the public is from United States, we will drop native.country, that will not bring a lot of information. And since we created a new variable "capital", we will drop "capital.gain" and "capital.loss".

In [ ]:
train = train.drop(columns = ['native.country', 'capital.gain', 'capital.loss'])
test = test.drop(columns = ['native.country', 'capital.gain', 'capital.loss'])

## 7. Data Preparation

In [ ]:
X = train.drop(columns = ['income', 'Id'])
y = train['income'].copy()

#split data into training and test dataset
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.20, random_state = 42)

databases = [X, X_train, X_val, test]

### 7.1. Rescaling

Due to the outliers, we will use RobustScaler to rescale the numeric data, since it is not sensitive to outliers.

In [ ]:
rs = RobustScaler()
columns = X_train.drop(columns = ['education.num']).select_dtypes(include = ['int64','float64']).columns
print(columns)
X_train[columns] = rs.fit_transform(X_train[columns].values)
X[columns] = rs.transform(X[columns].values)
X_val[columns] = rs.transform(X_val[columns].values)
test[columns] = rs.transform(test[columns].values)

In [ ]:
X_train.head()

### 7.2. Encoding

We will apply One-Hot Encoding to the following features:

* **OneHotEncoder**: sex, workclass, education, marital.status, occupation, relationship, race

education.num is already encoded.

In [ ]:
X_train.select_dtypes(include = 'object').apply(lambda x: x.unique().shape[0])

In [ ]:
onehot_columns = ['sex', 'workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race']
    
# OneHotEncoder
X = pd.get_dummies(X, prefix=onehot_columns, columns = onehot_columns, drop_first=True)
X_train = pd.get_dummies(X_train, prefix=onehot_columns, columns = onehot_columns, drop_first=True)
X_val = pd.get_dummies(X_val, prefix=onehot_columns, columns = onehot_columns, drop_first=True)
test = pd.get_dummies(test, prefix=onehot_columns, columns = onehot_columns, drop_first=True)

In [ ]:
X_train.head()

## 8. Machine Learning Modelling and Hyperparameter Tunning

In this section, different machine learning algorithms will be evaluated to classify the adult base. They will be:

* KNN
* Support Vector Classifier (SVC)
* Random Forest Classifier
* Extreme Gradient Boosting Classifier 
* Neural Network Classifier

### 8.1. KNN

From RandomizedSearchCV, we see that the best kNN estimator is the one with 14 neighbors, which has a cross-validation score of 0.861.

In [ ]:
# model definition
knn = KNeighborsClassifier()

# RandomizedSearchCV
parameters = {'n_neighbors' : np.arange(5, 31)}
knn_grid_cv = RandomizedSearchCV(knn, parameters, verbose = True, scoring='accuracy', cv = 5, n_iter = 50, n_jobs = -1, random_state = 42)
%timeit -n 1 -r 1 knn_grid_cv.fit(X_train, y_train)

print(f'Best estimator: {knn_grid_cv.best_estimator_}')
print(f'Best score: {knn_grid_cv.best_score_}')

# predict
y_predict_knn = knn_grid_cv.predict(X_val)

### 8.2. Support Vector Classifier (SVC)

From RandomizedSearchCV, we see that the best SVC estimator is the one with the following parameters:

* C = 8.985102040816326
* gamma = 'auto'

This set of parameters gives a cross-validation score of 0.869.

In [ ]:
# model definition
svc = SVC(random_state=42, probability=True)

# RandomizedSearchCV
parameters = {'C': np.linspace(1e-2, 20),
              'gamma': ['scale', 'auto']}
svc_grid_cv = RandomizedSearchCV(svc, parameters, verbose = True, scoring='accuracy', cv = 5, n_iter = 30, n_jobs = -1, random_state = 42)
%timeit -n 1 -r 1 svc_grid_cv.fit(X_train, y_train)

print(f'Best estimator: {svc_grid_cv.best_estimator_}')
print(f'Best score: {svc_grid_cv.best_score_}')

# predict
y_predict_svc = svc_grid_cv.predict(X_val)

### 8.3. Random Forest Classifier

From RandomizedSearchCV, we see that the best Random Forest Classifier estimator is the one with the following parameters:

* criterion = 'entropy'
* max_depth = 20
* n_estimators = 380

This set of parameters gives a cross-validation score of 0.861.

In [ ]:
# model definition
rfc = RandomForestClassifier(random_state=42)

# RandomizedSearchCV
parameters = {'n_estimators': np.arange(100, 500),
               'criterion': ['gini', 'entropy'],
               'max_depth': np.arange(1, 50),}
rfc_grid_cv = RandomizedSearchCV(rfc, parameters, verbose = True, scoring='accuracy', cv = 5, n_iter = 30, n_jobs = -1, random_state = 42)
%timeit -n 1 -r 1 rfc_grid_cv.fit(X_train, y_train)

print(f'Best estimator: {rfc_grid_cv.best_estimator_}')
print(f'Best score: {rfc_grid_cv.best_score_}')

# predict
y_predict_rfc = rfc_grid_cv.predict(X_val)

### 8.4. Extreme Gradient Boosting Classifier

From RandomizedSearchCV, we see that the best Extreme Gradient Boosting Classifier estimator is the one with the following parameters:

* n_estimators = 179
* learning_rate = 0.42914285714285716
* max_depth = 2
* reg_alpha = 0.013237491476160993
* reg_lambda = 3.169978726120134e-05

This set of parameters gives a cross-validation score of 0.871.

In [ ]:
# model definition
xgb = XGBClassifier(random_state=42)

# RandomizedSearchCV
parameters = {'n_estimators': np.arange(10, 500),
               'learning_rate': np.linspace(1e-3, 1),
               'max_depth': np.arange(1, 20),
               'reg_alpha': sp_loguniform(1e-14, 1e1),
               'reg_lambda': sp_loguniform(1e-14, 1e1),}
xgb_grid_cv = RandomizedSearchCV(xgb, parameters, verbose = True, cv = 5, n_iter = 30, n_jobs = -1, random_state = 42)
%timeit -n 1 -r 1 xgb_grid_cv.fit(X_train, y_train)

print(f'Best estimator: {xgb_grid_cv.best_estimator_}')
print(f'Best score: {xgb_grid_cv.best_score_}')

# predict
y_predict_xgb = xgb_grid_cv.predict(X_val)

### 8.5. Neural Network Classifier

From RandomizedSearchCV, we see that the best Neural Network Classifier estimator is the one with the following parameters:

* hidden_layer_sizes = (32, 128)
* alpha = 2.3488812958533118e-07
* learning_rate = constant

This set of parameters gives a cross-validation score of 0.854.

In [ ]:
# model definition
mlp = MLPClassifier(random_state=42, early_stopping=True)

# RandomizedSearchCV
parameters = {'hidden_layer_sizes': [(2 ** i, 2 ** j) for j in np.arange(5, 8) for i in np.arange(4, 7)],
               'alpha': sp_loguniform(1e-10, 1e-1),
               'learning_rate': ['constant','adaptive']}
mlp_grid_cv = RandomizedSearchCV(mlp, parameters, verbose = True, scoring='accuracy', cv = 5, n_iter = 30, n_jobs = -1, random_state = 42)
%timeit -n 1 -r 1 mlp_grid_cv.fit(X_train, y_train)

print(f'Best estimator: {mlp_grid_cv.best_estimator_}')
print(f'Best score: {mlp_grid_cv.best_score_}')

# predict
y_predict_mlp = mlp_grid_cv.predict(X_val)

It can be seen that Extreme Gradient Boosting Classifier returned the best cross validation score between all the classifiers. Thus, it will be used as the final model to predict the test dataset.

## 9. Models Performance

This section will evaluate each model trained with some specific metrics, such as precision, recall, f1-score and its respectives confusion matrix.

### 9.1. KNN


In [ ]:
# accuracy_score
dict_map = {'<=50K' : 0, '>50K' : 1}
accuracy_knn = accuracy_score(pd.Series(y_val).map(dict_map), pd.Series(y_predict_knn).map(dict_map))

print(f"Accuracy Score for KNN: {accuracy_knn}")

In [ ]:
#classification_report
print(classification_report(y_val, y_predict_knn))

In [ ]:
dict_map = {'<=50K' : 0, '>50K' : 1}
roc_auc_knn = roc_auc_score(pd.Series(y_val).map(dict_map), pd.Series(y_predict_knn).map(dict_map))

print(f"ROC-AUC Score for KNN: {roc_auc_knn}")

In [ ]:
#confusion_matrix
_ = mt.plot_confusion_matrix(y_val,y_predict_knn, normalize = False, figsize = (12,12))

### 9.2. Support Vector Classifier (SVC)

In [ ]:
# accuracy_score
dict_map = {'<=50K' : 0, '>50K' : 1}
accuracy_svc = accuracy_score(pd.Series(y_val).map(dict_map), pd.Series(y_predict_svc).map(dict_map))

print(f"Accuracy Score for SVC: {accuracy_svc}")

In [ ]:
#classification_report
print(classification_report(y_val, y_predict_svc))

In [ ]:
dict_map = {'<=50K' : 0, '>50K' : 1}
roc_auc_svc = roc_auc_score(pd.Series(y_val).map(dict_map), pd.Series(y_predict_svc).map(dict_map))

print(f"ROC-AUC Score for SVC: {roc_auc_svc}")

In [ ]:
#confusion_matrix
_ = mt.plot_confusion_matrix(y_val,y_predict_svc, normalize = False, figsize = (12,12))

### 9.3. Random Forest Classifier

In [ ]:
# accuracy_score
dict_map = {'<=50K' : 0, '>50K' : 1}
accuracy_rfc = accuracy_score(pd.Series(y_val).map(dict_map), pd.Series(y_predict_rfc).map(dict_map))

print(f"Accuracy Score for Random Forest Classifier: {accuracy_rfc}")

In [ ]:
#classification_report
print(classification_report(y_val, y_predict_rfc))

In [ ]:
dict_map = {'<=50K' : 0, '>50K' : 1}
roc_auc_rfc = roc_auc_score(pd.Series(y_val).map(dict_map), pd.Series(y_predict_rfc).map(dict_map))

print(f"ROC-AUC Score for Random Forest Classifier: {roc_auc_rfc}")

In [ ]:
#confusion_matrix
_ = mt.plot_confusion_matrix(y_val,y_predict_rfc, normalize = False, figsize = (12,12))

### 9.4. Extreme Gradient Boosting Classifier

In [ ]:
# accuracy_score
dict_map = {'<=50K' : 0, '>50K' : 1}
accuracy_xgb = accuracy_score(pd.Series(y_val).map(dict_map), pd.Series(y_predict_xgb).map(dict_map))

print(f"Accuracy Score for Extreme Gradient Boosting Classifier: {accuracy_xgb}")

In [ ]:
#classification_report
print(classification_report(y_val, y_predict_xgb))

In [ ]:
dict_map = {'<=50K' : 0, '>50K' : 1}
roc_auc_xgb = roc_auc_score(pd.Series(y_val).map(dict_map), pd.Series(y_predict_xgb).map(dict_map))

print(f"ROC-AUC Score for Extreme Gradient Boosting Classifier: {roc_auc_xgb}")

In [ ]:
#confusion_matrix
_ = mt.plot_confusion_matrix(y_val,y_predict_xgb, normalize = False, figsize = (12,12))

### 9.5. Neural Network Classifier

In [ ]:
# accuracy_score
dict_map = {'<=50K' : 0, '>50K' : 1}
accuracy_mlp = accuracy_score(pd.Series(y_val).map(dict_map), pd.Series(y_predict_mlp).map(dict_map))

print(f"Accuracy Score for Neural Network Classifier: {accuracy_mlp}")

In [ ]:
#classification_report
print(classification_report(y_val, y_predict_mlp))

In [ ]:
dict_map = {'<=50K' : 0, '>50K' : 1}
roc_auc_mlp = roc_auc_score(pd.Series(y_val).map(dict_map), pd.Series(y_predict_mlp).map(dict_map))

print(f"ROC-AUC Score for Neural Network Classifier: {roc_auc_mlp}")

In [ ]:
#confusion_matrix
_ = mt.plot_confusion_matrix(y_val,y_predict_mlp, normalize = False, figsize = (12,12))

### 9.6. Score Summarization

To compare each classifier, a consolidated table is elaborated with the pertinent metrics for the validation dataset metrics.

In [ ]:
metrics = [
            ['KNN', 0.86317, 0.82, 0.79, 0.80, 0.786],
            ['SVC', 0.87134, 0.84, 0.78, 0.80, 0.779],
            ['Random Forest Classifier', 0.86379, 0.83, 0.77, 0.80, 0.775],
            ['Extreme Gradient Boosting Classifier', 0.8736, 0.84, 0.80, 0.81, 0.797],
            ['Neural Network Classifier', 0.85549, 0.81, 0.77, 0.78, 0.765],
          ]

df_metrics = pd.DataFrame(metrics, columns = ['Classifier', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC Score'])

df_metrics

From the metrics in the validation dataset, it can be seen that the best model is the Extreme Gradient Boosting Classifier, which will then be used to predict values on the test dataset.

## 10. Fitting the model using all the data, best model and best parameters

In [ ]:
# model definition
best_model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.42914285714285716, max_delta_step=0, max_depth=2,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=179, n_jobs=0, num_parallel_tree=1, random_state=42,
              reg_alpha=0.013237491476160993, reg_lambda=3.169978726120134e-05,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

# fitting model
best_model.fit(X, y)

## 11. Predicting

In [ ]:
# predict
test['income'] = best_model.predict(test.drop(columns = 'Id'))

test.head()

## 9. Exporting result

In [ ]:
export_path = "submission.csv"
test[['Id', 'income']].to_csv(export_path, index = False)